In [ ]:
import datamule as dm
import asyncio
from datetime import datetime
import json
import os

class SECMonitor:
    def __init__(self):
        self.downloader = dm.Downloader()
        self.downloader.set_headers("Your Name your@email.com")
        
        # Create filings directory if it doesn't exist
        os.makedirs('filings', exist_ok=True)

    async def filing_callback(self, data):
        """Callback function to handle new filings"""
        try:
            filings = data['hits']['hits']
            for filing in filings:
                filing_info = {
                    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                    'company': filing['_source'].get('company_name'),
                    'form_type': filing['_source'].get('file_type'),
                    'filing_date': filing['_source'].get('file_date'),
                    'cik': filing['_source'].get('ciks')[0],
                    'accession_number': filing['_id']
                }
                print("\nNew Filing Detected:")
                print(json.dumps(filing_info, indent=2))
                
                # Save filing to disk
                await self.save_filing(filing_info)
        except Exception as e:
            print(f"Error processing filing callback: {str(e)}")

    async def save_filing(self, filing_info):
        """Save filing information to disk"""
        try:
            filename = f"filings/{filing_info['cik']}_{filing_info['filing_date']}.json"
            with open(filename, 'w') as f:
                json.dump(filing_info, f, indent=2)
        except Exception as e:
            print(f"Error saving filing: {str(e)}")

    async def monitor_filings(self, tickers=None, forms=None, interval=10):
        """
        Monitor SEC filings for specified tickers and forms.
        """
        try:
            print(f"\nStarting SEC filing monitor at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
            print(f"Monitoring tickers: {tickers}")
            print(f"Monitoring forms: {forms}")
            print(f"Check interval: {interval} seconds")
            print("\nWaiting for new filings...\n")

            monitoring_tasks = []
            for ticker in tickers:
                task = self.downloader.watch(
                    interval=interval,
                    silent=False,
                    form=forms,
                    ticker=ticker,
                    callback=self.filing_callback
                )
                monitoring_tasks.append(task)
            
            await asyncio.gather(*monitoring_tasks)
                
        except KeyboardInterrupt:
            print("\nMonitoring stopped by user")
        except Exception as e:
            print(f"\nError in monitoring: {str(e)}")

async def main():
    # Initialize monitor
    monitor = SECMonitor()
    
    # Configure monitoring parameters
    tickers = ['AAPL', 'MSFT', 'GOOGL']
    forms = ['8-K', '10-Q', '10-K']
    interval = 10  # seconds
    
    # Start monitoring
    await monitor.monitor_filings(
        tickers=tickers,
        forms=forms,
        interval=interval
    )

if __name__ == "__main__":
    asyncio.run(main())